In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

In [100]:
import time
import pickle
import pandas as pd
from IPython.display import display, Markdown

with open('/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl', 'rb') as f:
    lc_docs = pickle.load(f)

In [157]:
from llama_index.data_structs.node import Node, DocumentRelationship
from llama_index.node_parser import SimpleNodeParser
from llama_index import GPTVectorStoreIndex, ResponseSynthesizer
from llama_index import Document

from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain.chat_models import ChatOpenAI
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from algovate.llama.evaluate import run_experiment, grade_model_answer, grade_model_retrieval

from langchain.docstore.document import Document as LCDocument
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

##### Design a node without parsing

In [102]:
documents = []
for doc in lc_docs:
    documents.append(Document(text=doc.page_content, extra_info=doc.metadata))

In [103]:
parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(documents)

In [72]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.,))
max_input_size = 4096
num_output = 256
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

index = GPTVectorStoreIndex.from_documents(documents)

retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=1,
)

In [153]:
question = "give me a sample solidity contract to use Proof of Reserve Feeds?"
retrieved_node = retriever.retrieve(question)
retrieved_node[0].node.doc_id

'c71b2a0a-bf54-4fc0-be72-dc76009c97ce'

In [138]:
retrieved_node[0].node.get_type()

<NodeType.TEXT: '1'>

In [140]:
retrieved_node = retriever.retrieve("give me a sample solidity contract to use Proof of Reserve Feeds?")
retrieved_node[0].node.doc_id
retrieved_doc = None
for doc in documents:
    if retrieved_node[0].node.text in doc.text:
        retrieved_doc = LCDocument(page_content= doc.text)
splitter = TokenTextSplitter()
retrieved_docs = splitter.split_documents([retrieved_doc])

In [146]:
# t = ""
# for doc in retrieved_docs:
#     t += doc.page_content

# Markdown(t)

In [189]:
template = """
As an AI assistant specialized in ChainLink documentation, your mission is to effectively answer inquiries concerning ChainLink. Remember, if any questions deviate from this topic, kindly redirect the conversation by responding with, "Kindly focus your questions on ChainLink."

In the event you are uncertain about an answer, honesty is paramount. Please express that you don't have the necessary information instead of fabricating an answer.

Rely solely on the provided context for your responses. To aid your response generation, you have been provided with potential answers based on previous context. Write you answer in Markdown format.

Incoming Question: {question}

==========================
Provided Context: {context}
==========================
Suggested Answer: {answer}
Final Answer:
"""

summary_template = """
Given the following question, summarize the provided context to best answer it.

Incoming Question: {question}

Provided Context: {context}
"""

In [190]:
prompt_template = PromptTemplate(template=template, input_variables=["question", "context", "answer"])
summary_template = PromptTemplate(template=summary_template, input_variables=["question", "context"])

In [191]:
llm = ChatOpenAI(temperature=0.0)
chain = LLMChain(llm=llm, prompt=prompt_template)
summary_chain = LLMChain(llm=llm, prompt=summary_template)

In [192]:
summary = ""
for doc in retrieved_docs:
    p = prompt_template.format_prompt(question=question, context=doc.page_content, answer="")
    res = summary_chain.predict(context=doc.page_content, question=question)
    summary += res
    print(res)

The provided context explains the concept of Proof of Reserve Feeds, which provide the status of reserves for different assets. The context also describes the types of Proof of Reserve Feeds, including off-chain and cross-chain reserves, and the methods used to provide data for each type. To use Proof of Reserve Feeds in a Solidity contract, the contract should reference AggregatorV3Interface, which defines the external functions implemented by Data Feeds. However, the context does not provide a sample Solidity contract for using Proof of Reserve Feeds.
The provided context includes a sample Solidity contract that uses Proof of Reserve Feeds. The contract references the AggregatorV3Interface and includes a constructor that specifies the address of the feed. The contract also includes a function that returns the latest reserve value. The context includes a disclaimer that users should be careful with the configuration of the feeds used by their smart contracts and assume full responsibi

In [180]:
Markdown(retrieved_docs[0].page_content)

# Proof of Reserve Feeds

  * Overview
  * Types of Proof of Reserve Feeds
  * Off-chain reserves
  * Cross-chain reserves
  * Using Proof of Reserve Feeds

Chainlink Proof of Reserve Feeds provide the status of the reserves for
several assets. You can read these feeds the same way that you read other Data
Feeds. Specify the [Proof of Reserve Feed Address](/data-feeds/proof-of-reserve/addresses) that you want to read instead of specifying a Price Feed
address. See the [Using Data Feeds](/data-feeds/using-data-feeds) page to
learn more.

To find a list of available Proof of Reserve Feeds, see the [Proof of Reserve
Feed Addresses](/data-feeds/proof-of-reserve/addresses) page.

## Types of Proof of Reserve Feeds

Reserves are available for both cross-chain assets and off-chain assets. This
categorization describes the data attestation variations of Proof of Reserve
feeds and helps highlight some of the inherent market risks surrounding the
data quality of these feeds.

Reserves are available for both off-chain assets and cross-chain assets. Token
issuers prove the reserves for their assets through several different methods.

### Off-chain reserves

Off-Chain reserves are sourced from APIs through an [external
adapter](/chainlink-nodes/external-adapters/external-adapters).

Off-chain reserves provide their data using the following methods:

  * Third-party: An auditor, accounting firm, or other third party attests to reserves. This is done by combining both fiat and investment assets into a numeric value that is reported against the token.
  * Custodian: Reserves data are pulled directly from the bank or custodian. The custodian has direct access to the bank or vault holding the assets. Generally, this works when the underlying asset pulled requires no additional valuation and is simply reported on-chain.
  * ⚠️ Self-attested: Reserve data is read from an API that the token issuer hosts. Self-attested feeds carry additional risk.

### Cross-chain reserves

Cross-chain reserves are sourced from the network where the reserves are held.
Chainlink node operators can report cross-chain reserves by running an
[external adapter](/chainlink-nodes/external-adapters/external-adapters) and
querying the source-chain client directly. In some instances, the reserves are
composed of a dynamic list of IDs or addresses using a composite adapter.

Cross-chain reserves provide their data using the following methods:

  * Wallet address manager: The project uses the [IPoRAddressList](https://github.com/smartcontractkit/chainlink/blob/develop/contracts/src/v0.8/interfaces/PoRAddressList.sol) wallet address manager contract and self-attests to which addresses they own.
  * Wallet address: The project attests which addresses they own through a self-hosted API.

## Using Proof of Reserve Feeds

Read answers from Proof of Reserve Feeds the same way that you read other Data
Feeds. Specify the [Proof of Reserve Feed Address](/data-feeds/proof-of-reserve/addresses) that you want to read instead of specifying a Price Feed
address. See the [Using Data Feeds](/data-feeds/using-data-feeds) page to
learn more.

Using Solidity, your smart contract should reference
[`AggregatorV3Interface`](https://github.com/smartcontractkit/chainlink/blob/master/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol),
which defines the external functions implemented by Data Feeds.


    // SPDX-License-Identifier: MIT
    pragma solidity ^0.8.7;
    
    import "@chainlink/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol";
    
    contract